<a href="https://colab.research.google.com/github/JaswanthMannem/CREWAI/blob/main/crewai_2_customer_suppourt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_huggingface langchain-openai

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI

In [5]:

from google.colab import userdata
key=userdata.get('OPENAI_API_KEY')


In [6]:

import os

openai_api_key = key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [7]:
support_agent = Agent(
    role="Senior Support Representative",
    goal="Be the most friendly and helpful "
        "support representative in your team",
    backstory=(
        "You work at crewAI (https://crewai.com) and "
        " are now working on providing "
        "support to {customer}, a super important customer "
        " for your company."
        "You need to make sure that you provide the best support!"
        "Make sure to provide full complete answers, "
        " and make no assumptions."
    ),
    allow_delegation=False,
    verbose=True,
    llm=ChatOpenAI(openai_api_key=openai_api_key)
)

In [8]:
support_quality_assurance_agent = Agent(
    role="Support Quality Assurance Specialist",
    goal="Get recognition for providing the "
    "best support quality assurance in your team",
    backstory=(
        "You work at crewAI (https://crewai.com) and "
        "are now working with your team "
        "on a request from {customer} ensuring that "
        "the support representative is "
        "providing the best support possible.\n"
        "You need to make sure that the support representative "
        "is providing full"
        "complete answers, and make no assumptions."
    ),
    verbose=True,
    llm=ChatOpenAI(openai_api_key=openai_api_key)
)

In [9]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

In [10]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [11]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
        "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
        "Make sure to use everything you know "
        "to provide the best support possible."
        "You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
        "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
        "leaving no questions unanswered, and maintain a helpful and friendly "
        "tone throughout."
    ),
    tools=[docs_scrape_tool],
    agent=support_agent,
)

In [12]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
        "high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
        "thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
        "ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
        "relevant feedback and improvements.\n"
        "Don't be too formal, we are a chill and cool company "
        "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

In [13]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  embedder={
            "provider": "huggingface",
            "config": {"model": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"},
        },
  verbose=2,
  memory=True
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to provide detailed and accurate guidance to Andrew Ng from DeepLearningAI regarding setting up a Crew and adding memory to it. I should use the tool available to gather information before providing a complete response.

Action: Read website content
Action Input: {} 

crewAI
 crewAI
 Initializing search
 crewAI
 crewAI
 crewAI
 Home
 Getting Started
 Getting Started
 Installing CrewAI
 Starting a new CrewAI project
 Core Concepts
 Core Concepts
 Agents

In [15]:
from IPython.display import Markdown
Markdown(result)

Hello Andrew Ng from DeepLearningAI,

Thank you for reaching out with your question about setting up a Crew and adding memory to it. I understand the importance of this task for your work, and I am here to provide you with detailed guidance on how to achieve this.

To set up a Crew and add memory to it, you can follow the steps outlined below:

1. Log in to your crewAI account and navigate to the "Crews" section.
2. Click on the option to create a new Crew.
3. Name your Crew to make it easily identifiable.
4. Once the Crew is created, access the settings menu within the Crew dashboard.
5. Look for the option to "Add Memory" or "Memory Settings" within the settings menu.
6. Click on the "Add Memory" option and allocate the desired memory capacity to your Crew.

By following these steps, you should be able to successfully set up a Crew and add memory to it. If you encounter any issues or need further assistance, please don't hesitate to reach out to our support team for additional help.

I hope this response addresses your inquiry thoroughly. If you have any more questions or need clarification on any part of this process, feel free to let me know. We are here to support you every step of the way.

Best regards,
[Your Name]
Senior Support Representative
crewAI